# Segmenting And Clustering Neighborhoods in Torornto Part-2:

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries Imported:')

Libraries Imported:


##### Get the HTML of the Wiki page, convert into a table with help of read_html (read HTML tables into a list of DataFrame objects), remove cells with a borough that is Not assigned.



In [4]:
List='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(List)
df=pd.read_html(source.content,header=0)[0]
df_new=df[df.Borough!='Not assigned']
df_new.head(8)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


##### Find whether there is 'Not Assigned' in Neighbourhood:

In [5]:
df_new.loc[df_new.Neighbourhood=='Not assigned']

,Postal Code,Borough,Neighbourhood


##### Since in the above data frame we didnt got any name of Neighbourhood that is not assigned, therefore we need not to be run the next command cell:

##### Skip the next code cell after the below markdown cell if above condition is satisfied:

##### If we have Neighbourhood Not assigned, we change it with the value of Borough

In [ ]:
df_new.Neighbourhood.replace('Not assigned',df_new.Borough,inplace=True)
df_new.head(8)

##### To Group Neighbourhood with same Postcode:

In [6]:
df_toronto=df_new.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(lambda x:','.join(x))
df_toronto=df_toronto.reset_index()
df_toronto.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_toronto.shape

(103, 3)

##### Lets get the latitude and longitude of each neighborhood:

#### Lets use the CSV file:

In [8]:
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
df_geo.shape

(103, 3)

#### Since both df_toronto and df_geo had same shape, so lets join them, Joining them on the basis of Postal Code:

In [10]:
df_toronto=df_toronto.join(df_geo.set_index('Postal Code'), on= 'Postal Code')
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
df_toronto.shape

(103, 5)